# Data Assimilation with GPS Data

This project demonstrates the Ensemble Kalman Filter (EnKF) using real GPS data procured from two phones measuring the location of the same thing (me driving my car around a parking lot). It is meant as a hands-on experience in overcoming each obstacle of real-world data assimilation, including asynchronous and irregularly-spaced observations that don't directly map to the timing of the state model. The second issue is determining the accuracy of the model - for my case, I was able to make an approximate "true" solution by mapping out my route using a tool that generates a route using coordinates. 

<img src = "../img/routes.png" height = 300 width = 300/>